In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})

import numpy as np
import itertools
import glob

import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
from parse_cube_files import CUBE
import alchemy_tools as at


In [2]:
# linear fit through origin
def linear_m(x, y):
    xy_av = np.dot(x,y)
    x2_av = np.dot(x,x)
    return(xy_av/x2_av)


# Import data into cube objects

In [3]:
paths = glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/cube-files/*.cube')
paths.sort()

cube_obj = []
for p in paths:
    cube_obj.append(CUBE(p))

In [4]:
# uniform electron gas
path_ueg = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/ueg/ve_00.cube'
cube_obj.insert(0, CUBE(path_ueg))

# Example 1D

In [72]:
# fft of all densities
# store coefficients for all lambda-values

# fourier coefficients for every lambda value
fft_freqs = np.empty((len(cube_obj), cube_obj[0].data_scaled.shape[0]), dtype='complex128')

# density before scaling
dens_unscaled = np.zeros((len(cube_obj), len(cube_obj[0].data_scaled)))

lam_val = np.array([0, 4, 6, 8, 10, 12, 14, 15, 16, 18, 20, 22, 23, 24, 26, 28, 29, 30, 32, 34, 36, 38])/38
for i in range(len(cube_obj)):
    ve_ex = cube_obj[i]
    pr = ve_ex.project((1,2))
    dens_unscaled[i] = pr

    # fft for one specific lambda value
    fft = np.fft.fft(pr)
    fft_freqs[i] = fft

In [66]:
# rescaling of frequencies

fft_freqs_scaled = fft_freqs.copy()

# do not rescale the coefficients at lambda = 1
end_idx = len(lam_val)-1

scaled_freqs = range(1,5)

for f in scaled_freqs:
    # scale imaginary part
    m_imag = linear_m(lam_val[:end_idx], fft_freqs.imag[:end_idx,f])
    fft_freqs_scaled.imag[:end_idx,f] = lam_val[:end_idx]*m_imag
    # scale real part
    m_real = linear_m(lam_val[:end_idx], fft_freqs.real[:end_idx,f])
    fft_freqs_scaled.real[:end_idx,f] = lam_val[:end_idx]*m_real
    

In [67]:
# plot frequencies before and after rescaling

fig_coeff, ax_coeff = plt.subplots(1,3)
plt.rcParams.update({'font.size': 18})
fig_coeff.suptitle(r'Fourier coefficients of the 5 lowest frequencies as a function of $\lambda$' + '\n' + 'for the 1D-projected densities of dsgdb9nsd_001212')
plt.rcParams.update({'font.size': 22})

# same colors for fit and original data
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

for i in range(0,5):
    # original values
    ax_coeff[0].plot(lam_val, fft_freqs.real[:,i], '-o', label = r'$\nu = ${}'.format(i), color=colors[i])
    ax_coeff[1].plot(lam_val, fft_freqs.imag[:,i], '-o', label = r'$\nu = ${}'.format(i), color=colors[i])
    ax_coeff[2].plot(lam_val, np.abs(fft_freqs[:,i]), '-o', label = r'$\nu = ${}'.format(i), color=colors[i])
    # after rescaling
    ax_coeff[0].plot(lam_val, fft_freqs_scaled.real[:,i], '--', color=colors[i])
    ax_coeff[1].plot(lam_val, fft_freqs_scaled.imag[:,i], '--', color=colors[i])
    ax_coeff[2].plot(lam_val, np.abs(fft_freqs_scaled[:,i]), '--', color=colors[i])

ax_coeff[0].set_xlabel(r'$\lambda$')
ax_coeff[0].set_ylabel(r'$c_{\nu, real}(\lambda)$')
ax_coeff[0].legend()

ax_coeff[1].set_xlabel(r'$\lambda$')
ax_coeff[1].set_ylabel(r'$c_{\nu, im}(\lambda)$')
ax_coeff[1].legend()

ax_coeff[2].set_xlabel(r'$\lambda$')
ax_coeff[2].set_ylabel(r'$|c_{\nu}(\lambda)|$')

Text(0, 0.5, '$|c_{\\nu}(\\lambda)|$')

In [68]:
rescaled_dens = np.zeros((len(cube_obj), len(cube_obj[0].data_scaled)))

for i in range(len(cube_obj)):

    ve_ex = cube_obj[i]
    pr = ve_ex.project((1,2))
    
    ifft = np.fft.ifft(fft_freqs_scaled[i])
    if not np.allclose(np.abs(ifft), np.abs(ifft.real)):
        print(max(np.abs(ifft) - np.abs(ifft.real)))
    rescaled_dens[i] = ifft.real

4.360595673352785e-06
3.7755281011520037e-05
6.416373028950151e-05
1.7859890254379174e-05
2.0778774865143346e-05
1.685419595831572e-05
4.1873553230625804e-05
3.563925757449349e-05
3.144131658147642e-05
6.229712086479322e-05
5.480544719273228e-05
5.5948641548980715e-05
4.309352345907569e-05
3.087351369801539e-05
1.8205497506279533e-05
5.4883729209892484e-05
0.00015518850438134973
0.00010866984782700179
5.597431262313948e-05
0.00020802540353079095
0.0016909305290532563


In [69]:
# plot density after and before rescaling

# same colors for fit and original data
from matplotlib import cm
colors = cm.jet(np.linspace(0, 1, len(cube_obj)))

x = np.linspace(0, 20, cube_obj[0].data_scaled.shape[0])

fig, ax = plt.subplots(1,1)

for i in range(len(cube_obj)):

    ve_ex = cube_obj[i]
    pr = ve_ex.project((1,2))
    
    ifft = np.fft.ifft(fft_freqs_scaled[i])
    
    ax.plot(x, rescaled_dens[i], '--', color = colors[i])
    ax.plot(x, pr, color = colors[i])
ax.set_xlabel('Cell coordinate (Ang)')
ax.set_ylabel('1D-Projected Density')

Text(0, 0.5, '1D-Projected Density')

In [88]:
# plot density at box edges as a function of lambda after and before rescaling

x = np.linspace(0, 20, cube_obj[0].data_scaled.shape[0])

fig, ax = plt.subplots(1,1)

ax.plot(lam_val, dens_unscaled[:,0], '-o', label = 'unscaled, left')
ax.plot(lam_val, rescaled_dens[:,0], '-o', label = 'scaled, left')
ax.legend()
fig, ax = plt.subplots(1,1)
ax.plot(lam_val, dens_unscaled[:,dens_unscaled.shape[1]-1], '-o', label = 'unscaled, rigth')
ax.plot(lam_val, rescaled_dens[:,rescaled_dens.shape[1]-1], '-o', label = 'scaled, right')

ax.legend()
# for i in range(len(cube_obj)):

#     rescaled_dens[i][0]
    
#     ax.plot(x, ifft.real, '--', color = colors[i])
#     ax.plot(x, pr, color = colors[i])
# ax.set_xlabel('Cell coordinate (Ang)')
# ax.set_ylabel('1D-Projected Density')

In [85]:
dens_unscaled[:,dens_unscaled.shape[1]-1]

array([ 2.17561191e-01,  1.95191796e-01,  1.84142293e-01,  1.68924052e-01,
        1.59537668e-01,  1.48212904e-01,  1.36417398e-01,  1.31125364e-01,
        1.26509477e-01,  1.15690073e-01,  1.05859624e-01,  9.47912460e-02,
        8.81823277e-02,  8.19872962e-02,  6.91103131e-02,  5.80973633e-02,
        5.67101003e-02,  4.99717361e-02,  3.55992734e-02,  2.18056533e-02,
        5.33349877e-03, -2.32032080e-08])

In [71]:
dens_unscaled

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])